In [1]:
import requests
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime, timezone
import time

In [2]:
# Measure exec time
start = datetime.now()

In [3]:
### Calculation exchange rate of reference to further math on pct_change:
def exchange_reference(df, date_reference):
    
    memory = {}
        
    for currency_code in df["currency_code"].unique():
        
        if currency_code != "VED":
            date_reference = date_reference
        else:
            date_reference = "2021-10-06"

    
        ## Select currency for calculations:
        df_exchange = df[df['currency_code'] == currency_code ]

        ## Select date to calculate reference

        df_date_reference = df_exchange.loc[(df_exchange.date).dt.date == pd.Timestamp(date_reference)]

        ### Averare exchange rate for day of reference:

        implicit_exchange_reference = df_date_reference.implicit_exchange.mean()
        
        memory[currency_code] = implicit_exchange_reference

    ### Percentual Variation

#     equation = ((df_exchange.implicit_exchange / implicit_exchange_reference)-1)*100
    
    
    return memory

In [4]:
from sqlalchemy import Column, Integer, Float, String, DateTime,Boolean, ForeignKey
from sqlalchemy.orm import declarative_base
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

path_to_currencies = "sqlite:///data/CurrenciesDataBase.db"
df_market = pd.read_sql('currencies_vs_btc', path_to_currencies)

### The pct calculations are not properly saved on the dataset unless I upload one previously saved. Why?
# The pipeline defined solve the issue by saving the calculated data on a new dataset but the calculations
# are done in the full dataset everytime the code runs.
# This practice is not scalable

# The pct could be calculated on the web interface for a given date selected by the users but this implies 
# more processing time

In [5]:
df = df_market.groupby(by=["date","currency_code"]).mean()
df.reset_index(inplace=True)

### Dataset with implicit exchange of reference by currency:
### The exchange of reference corresponds to the first datapoint on the dataset
date_reference = "2021-10-03"

exchange = exchange_reference(df, date_reference)
exchange = pd.DataFrame.from_dict(exchange, orient="index")

exchange.head()

/home/becode/.local/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)


,0
ADA,0.448517
AED,3.677105
AOA,727.218777
ARS,179.293196
AUD,1.417418


In [6]:
df['pct'] = df.apply(lambda x : ((x['implicit_exchange']/exchange.loc[x["currency_code"]])-1)*100, axis = 1 )

### Drop Null values:

df.dropna(subset=['pct'], inplace=True)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12057 entries, 0 to 13048
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   date               12057 non-null  datetime64[ns]
 1   currency_code      12057 non-null  object        
 2   time_stamp         12057 non-null  float64       
 3   volume_btc         12057 non-null  float64       
 4   avg_24h            12057 non-null  float64       
 5   avg_12h            11025 non-null  float64       
 6   avg_6h             9807 non-null   float64       
 7   avg_1h             5887 non-null   float64       
 8   altcoin            12057 non-null  float64       
 9   avg_24h_usd        12057 non-null  float64       
 10  implicit_exchange  12057 non-null  float64       
 11  pct                12057 non-null  float64       
dtypes: datetime64[ns](1), float64(10), object(1)
memory usage: 1.2+ MB


In [8]:
### Saving on DB Version 2

engine = create_engine('sqlite:///data/CurrenciesDataBase_V2.db')
print(type(engine))
df.to_sql(name="currencies_vs_btc", con=engine, if_exists="append", index=False)

<class 'sqlalchemy.engine.base.Engine'>


In [9]:
end = dt.datetime.now()
print("Last Execution: ")
print(str(end))
print('Completed')

Last Execution: 
2022-02-01 22:39:55.786542
Completed


In [10]:
print("Execution time:", end-start)

Execution time: 0:00:07.209390
